# Reservoir Management

## Problem Description

From [Kowalik P, Rzemieniak M. Binary Linear Programming as a Tool of Cost Optimization for a Water Supply Operator. Sustainability. 2021 13(6):3470.](https://doi.org/10.3390/su13063470)

> The water supply system under consideration is that of a water supply operator based in a town with a population of about 25,000 inhabitants, located in Eastern Poland. **The main parts of the system are wells, pumps, a reservoir tank, and the distribution pipeline network.**

> Supplied water is groundwater pumped from 7 wells. The capacities and values of the electric power of the pumps are presented in Table 1. **Water is pumped from the wells to a single reservoir tank with the capacity of Vmax = 1500 m3 (the maximal volume of stored water)**.

> **The demand for water varies over time**. The outflow of water from the reservoir tank via the distribution network to customers is a continuous process. For practical reasons, **predictions of demand are made for 24 one-hour timeslots**.

> Controlling the pumps must obey the following requirements. **The pumps can operate with their nominal capacities only**, and the amount of water pumped by any pump depends on the time of operation only. **Each pump must operate for at least one hour per day.** Additionally, **at least one well and the pump integrated with it must be kept as a reserve at any moment of the day**. **The water inside the tank should be replaced at least once per day**. During standard operational conditions, **the volume of water in the reservoir tank cannot be less than Vmin = 523.5 m3**. It is the firefighting reserve, which is kept in order to satisfy an extra demand when a fire is extinguished by using water supplied from hydrants.

> ... the supplier of electric power does not use the same rate per MWh in its pricing policy all day long. Instead, it uses three tariff levels ...

## Problem instance data

From [Kowalik P, Rzemieniak M. Binary Linear Programming as a Tool of Cost Optimization for a Water Supply Operator. Sustainability. 2021 13(6):3470.](https://doi.org/10.3390/su13063470)

In [1]:
%%file pumps.csv
id capacity power_consumption
1 75 15
2 133 37
3 157 33
4 176 33
5 59 22
6 69 33
7 120 22

Overwriting pumps.csv


In [2]:
%%file tariff.csv
start end price
16 21 336.00
7 13 283.00
0 7 169.00
13 16 169.00
21 24 169.00

Overwriting tariff.csv


In [3]:
%%file schedule.csv
time water_demand power_price
1 44.62 169
2 31.27 169
3 26.22 169
4 27.51 169
5 31.50 169
6 46.18 169
7 69.47 169
8 100.36 283
9 131.85 283
10 148.51 283
11 149.89 283
12 142.21 283
13 132.09 283
14 129.29 169
15 124.06 169
16 114.68 169
17 109.33 336
18 115.76 336
19 126.95 336
20 131.48 336
21 138.86 336
22 131.91 169
23 111.53 169
24 70.43 169

Overwriting schedule.csv


In [4]:
%%file reservoir.csv
Vmin Vmax Vinit
523.5 1500 550

Overwriting reservoir.csv


## Load data

In [5]:
import itertools as it
import functools as ft
import collections as coll
import random, json, string, sys, pickle, shutil, string

import numpy as np
import pandas as pd
import networkx as nx
from cytoolz import curried as tl
import janitor

from icecream import ic
from prettyprinter import pprint as pp, install_extras

import dimod
from dwave import system as dw

In [6]:
pumps = pd.read_csv("pumps.csv", sep=' ').set_index('id', drop=False)
schedule = pd.read_csv("schedule.csv", sep=' ').set_index('time', drop=False)
tariff = pd.read_csv("tariff.csv", sep=' ')
reservoir = pd.read_csv("reservoir.csv", sep=' ').loc[0]

## Define domain language

In [7]:
Sum = dimod.quicksum

### Inputs (Parameters)

In [8]:
def capacity(pump):
    return pumps.capacity[pump]

def power_consumption(pump):
    return pumps.power_consumption[pump]

def power_price(time):
    return schedule.power_price[time]/1000

def water_demand(time):
    return schedule.water_demand[time]

### Outputs (Variables)

In [9]:
def is_running(pump, time):
    return dimod.Binary(f"pump{pump}_time{time}")

def reservoir_volume(time): 
    return (
        dimod.Real(f"volume_time{time}")
        if time >= schedule.time.min()
        else reservoir.Vinit
    )

### Derived terms

In [10]:
def reservoir_inflow(time):
    return Sum(
            capacity(pump) * is_running(pump, time)
            for pump in pumps.id
        )

def power_used(time):
    return Sum(
            power_consumption(pump) * is_running(pump, time)
            for pump in pumps.id
        )

## Construct model

In [11]:
model = dimod.CQM()

### Define objective

#### *Minimize power costs*

In [12]:
model.set_objective(
    Sum(
        power_price(time) * power_used(time)
        for time in schedule.time
    )
)

### Add constraints

#### *Each pump must operate for at least one hour per day*

In [13]:
for pump in pumps.id:
    model.add_constraint(
        Sum(
            is_running(pump, time) for time in schedule.time
        ) >= 1,
        f"pump{pump}_on_at_least_1h_per_day")

#### *At least one well and the pump integrated with it must be kept as a reserve at any moment of the day*

In [14]:
for time in schedule.time:
    model.add_constraint(
        Sum(
            is_running(pump, time) for pump in pumps.id
        ) <= pumps.shape[0] - 1,
        f"at_least_one_pump_in_reserve_at_time{time}" )

#### *The water inside the tank should be replaced at least once per day* ??? (Not addressed in paper. Meaning?)

In [15]:
# model.add_constraint(reservoir_volume(schedule.time.max()) >= reservoir.Vinit, f"water_in_tank_replaced");

#### *The volume of water in the reservoir tank must always be between Vmin and Vmax*

In [16]:
for time in schedule.time:
    model.add_constraint(
        reservoir_volume(time-1) + reservoir_inflow(time) - reservoir_volume(time) == water_demand(time),
        #f"reservoir volume at time_{time} = volume_at time_{time-1} + inflow at time_{time} - water demand at time_{time}"
        f"volume_at_time{time}"
    )
    model.add_constraint(
        reservoir_volume(time) >= reservoir.Vmin,
        f"reservoir_volume_at_time{time}_at_least_Vmin"
    )
    model.add_constraint(
        reservoir_volume(time) <= reservoir.Vmax,
        f"reservoir_volume_at_time{time}_at_most_Vmax"
    )

## Solve Model

In [17]:
sampler = dw.LeapHybridCQMSampler()

In [18]:
%%time
samples = sampler.sample_cqm(model, time_limit=20)
samples.resolve()

CPU times: user 868 ms, sys: 42.4 ms, total: 910 ms
Wall time: 21.1 s


In [19]:
feasible = samples.filter(lambda d: d.is_feasible)

In [20]:
feasible.to_pandas_dataframe(True)

,sample,energy,num_occurrences,is_satisfied,is_feasible
0,"{'pump1_time1': 1.0, 'pump1_time10': 0.0, 'pum...",103.256,1,"[True, True, True, True, True, True, True, Tru...",True
1,"{'pump1_time1': 1.0, 'pump1_time10': 0.0, 'pum...",96.999,1,"[True, True, True, True, True, True, True, Tru...",True
2,"{'pump1_time1': 0.0, 'pump1_time10': 0.0, 'pum...",97.713,1,"[True, True, True, True, True, True, True, Tru...",True
3,"{'pump1_time1': 0.0, 'pump1_time10': 0.0, 'pum...",95.027,1,"[True, True, True, True, True, True, True, Tru...",True
4,"{'pump1_time1': 0.0, 'pump1_time10': 0.0, 'pum...",180.713,1,"[True, True, True, True, True, True, True, Tru...",True
5,"{'pump1_time1': 0.0, 'pump1_time10': 0.0, 'pum...",102.480,1,"[True, True, True, True, True, True, True, Tru...",True
6,"{'pump1_time1': 0.0, 'pump1_time10': 0.0, 'pum...",103.776,1,"[True, True, True, True, True, True, True, Tru...",True
7,"{'pump1_time1': 0.0, 'pump1_time10': 0.0, 'pum...",102.826,1,"[True, True, True, True, True, True, True, Tru...",True


## Inspect model

In [21]:
lp_dump = dimod.lp.dumps(model)

In [22]:
print(lp_dump)

Minimize
 obj: + 2.535 pump1_time1 + 6.253 pump2_time1 + 5.577 pump3_time1 
 + 5.577 pump4_time1 + 3.7180000000000004 pump5_time1 + 5.577 pump6_time1 
 + 3.7180000000000004 pump7_time1 + 2.535 pump1_time2 + 6.253 pump2_time2 
 + 5.577 pump3_time2 + 5.577 pump4_time2 + 3.7180000000000004 pump5_time2 
 + 5.577 pump6_time2 + 3.7180000000000004 pump7_time2 + 2.535 pump1_time3 
 + 6.253 pump2_time3 + 5.577 pump3_time3 + 5.577 pump4_time3 
 + 3.7180000000000004 pump5_time3 + 5.577 pump6_time3 
 + 3.7180000000000004 pump7_time3 + 2.535 pump1_time4 + 6.253 pump2_time4 
 + 5.577 pump3_time4 + 5.577 pump4_time4 + 3.7180000000000004 pump5_time4 
 + 5.577 pump6_time4 + 3.7180000000000004 pump7_time4 + 2.535 pump1_time5 
 + 6.253 pump2_time5 + 5.577 pump3_time5 + 5.577 pump4_time5 
 + 3.7180000000000004 pump5_time5 + 5.577 pump6_time5 
 + 3.7180000000000004 pump7_time5 + 2.535 pump1_time6 + 6.253 pump2_time6 
 + 5.577 pump3_time6 + 5.577 pump4_time6 + 3.7180000000000004 pump5_time6 
 + 5.577 pump6_

In [23]:
with open("reservoir.lp", "w") as f:
    print(lp_dump, file=f)